In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import plotly.express as px
import datetime
import bs4 as bs

In [3]:
import statsmodels.api as sm


In [6]:
sp500_aapl = yf.download('WMT', start='2020-01-13', end ='2023-05-05')
wmt_df = pd.DataFrame(sp500_aapl)
wmt_df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-13,38.793331,38.849998,38.456669,38.626667,36.142704,18337800
2020-01-14,38.490002,38.736668,38.376667,38.726665,36.236275,19757400
2020-01-15,38.209999,38.623333,38.173332,38.426666,35.955570,22362600
2020-01-16,38.500000,38.643333,38.426666,38.633331,36.148949,16108200
2020-01-17,38.683334,38.766666,38.213333,38.320000,35.855762,30135000
...,...,...,...,...,...,...
2023-04-28,50.430000,50.723331,50.060001,50.323334,49.601517,14088600
2023-05-01,50.346668,50.846668,50.310001,50.529999,49.805222,11409600
2023-05-02,50.490002,50.496666,49.820000,50.393333,49.670517,11329500


In [143]:
wmt_df["Volt"] = np.log(wmt_df["High"]/wmt_df["Low"])
wmt_df["Return"] = wmt_df["Close"].pct_change().abs() 
wmt_df["illq"] = wmt_df["Return"]/wmt_df["Volume"]*10**7
wmt_df['spread'] = wmt_df["High"] - wmt_df["Low"]
wmt_df

,Open,High,Low,Close,Adj Close,Volume,Volt,Return,illq,spread
Date,,,,,,,,,,
2020-01-13,38.793331,38.849998,38.456669,38.626667,36.142704,18337800,0.010176,NaN,NaN,0.393330
2020-01-14,38.490002,38.736668,38.376667,38.726665,36.236275,19757400,0.009337,0.002589,0.001310,0.360001
2020-01-15,38.209999,38.623333,38.173332,38.426666,35.955570,22362600,0.011719,0.007747,0.003464,0.450001
2020-01-16,38.500000,38.643333,38.426666,38.633331,36.148949,16108200,0.005623,0.005378,0.003339,0.216667
2020-01-17,38.683334,38.766666,38.213333,38.320000,35.855762,30135000,0.014376,0.008110,0.002691,0.553333
...,...,...,...,...,...,...,...,...,...,...
2023-04-28,50.430000,50.723331,50.060001,50.323334,49.601517,14088600,0.013164,0.002708,0.001922,0.663330
2023-05-01,50.346668,50.846668,50.310001,50.529999,49.805222,11409600,0.010611,0.004107,0.003599,0.536667
2023-05-02,50.490002,50.496666,49.820000,50.393333,49.670517,11329500,0.013491,0.002705,0.002387,0.676666


In [8]:
from arch import arch_model

# Загрузите данные
# Предположим, что ваш DataFrame называется df и содержит столбцы 'high', 'open', 'low', 'close'

# Вычислите ежедневные доходности
daily_returns = wmt_df['Close'].pct_change()

# Рассчитайте первую разность ежедневных доходностей
first_diff_returns = daily_returns.diff().dropna()

# Оцените модель GARCH(1,1)
garch_model = arch_model(first_diff_returns, vol='GARCH', p=1, q=1)
results = garch_model.fit()

# Выведите результаты оценки модели
print(results.summary())

# Получите оцененную условную волатильность
conditional_volatility = results.conditional_volatility

# Выведите условную волатильность
print(conditional_volatility)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 11226318.794105036
Iteration:      2,   Func. Count:     17,   Neg. LLF: -2121.9621860437555
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2121.962188139154
            Iterations: 6
            Function evaluations: 17
            Gradient evaluations: 2
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                  Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                2121.96
Distribution:                  Normal   AIC:                          -4235.92
Method:            Maximum Likelihood   BIC:                          -4217.03
                                        No. Observations:                  832
Date:                Sun, May 05 2024   Df Residuals:                      831
Time:        

C:\Users\casov\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0005338. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [9]:
price = np.array(["Price",
                  wmt_df["Close"].mean(),
                  wmt_df["Close"].std(),
                  wmt_df["Close"].min(),
                  wmt_df["Close"].quantile(0.25),
                  wmt_df["Close"].quantile(0.5),
                  wmt_df["Close"].quantile(0.75),
                  wmt_df["Close"].max()
 ])
volume = np.array(["Volume",
                  wmt_df["Volume"].mean(),
                  wmt_df["Volume"].std(),
                  wmt_df["Volume"].min(),
                  wmt_df["Volume"].quantile(0.25),
                  wmt_df["Volume"].quantile(0.5),
                  wmt_df["Volume"].quantile(0.75),
                  wmt_df["Volume"].max()
 ])
Volt = np.array(["Volt",
                  wmt_df["Volt"].mean(),
                  wmt_df["Volt"].std(),
                  wmt_df["Volt"].min(),
                  wmt_df["Volt"].quantile(0.25),
                  wmt_df["Volt"].quantile(0.5),
                  wmt_df["Volt"].quantile(0.75),
                  wmt_df["Volt"].max()
 ])
vol = np.array([conditional_volatility])
Gvolt= np.array(["GVolt",
                  vol.mean(),
                  vol.std(),
                  vol.min(),
                  np.quantile(vol,0.25),
                  np.quantile(vol,0.5),
                  np.quantile(vol,0.75),
                  vol.max()
 ])
illq= np.array(["Illiquidity",
                  wmt_df["illq"].mean(),
                  wmt_df["illq"].std(),
                  wmt_df["illq"].min(),
                  wmt_df["illq"].quantile(0.25),
                  wmt_df["illq"].quantile(0.5),
                  wmt_df["illq"].quantile(0.75),
                  vol.max()
 ])

print(price,volume,Volt,Gvolt,illq,sep="\n")
########################################

['Price' '45.821622692995504' '3.4839774585742753' '34.68333435058594'
 '43.7983341217041' '46.6349983215332' '48.22999954223633'
 '53.290000915527344']
['Volume' '24713566.90647482' '13382844.871830858' '8777400' '17102775.0'
 '20771250.0' '28094100.0' '132940200']
['Volt' '0.018357345878141435' '0.01151537934016121'
 '0.005208675407843273' '0.01161203733850744' '0.015567572492206014'
 '0.02157847547319062' '0.11208019689050416']
['GVolt' '0.020834573084505016' '0.010138350255716523'
 '0.013816903186414197' '0.015934625791850537' '0.018106498468718514'
 '0.021257988826778627' '0.11981852232544699']
['Illiquidity' '0.003963675710612197' '0.003161313759225236'
 '2.0210138298736176e-05' '0.0016318605928406728' '0.0033494969759637395'
 '0.00558522128004357' '0.11981852232544699']


In [126]:
gvolt  = conditional_volatility.loc["2020-02-15":"2022-08-27"]
gvolt

Date
2020-02-18    0.015657
2020-02-19    0.015762
2020-02-20    0.020529
2020-02-21    0.020036
2020-02-24    0.018582
                ...   
2022-08-22    0.025569
2022-08-23    0.022652
2022-08-24    0.021196
2022-08-25    0.019843
2022-08-26    0.018141
Name: cond_vol, Length: 638, dtype: float64

In [127]:
wmt = wmt_df.loc["2020-02-15":"2022-08-27"]
wmt["gvolt"] = gvolt
wmt

C:\Users\casov\AppData\Local\Temp\ipykernel_10472\2986010439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wmt["gvolt"] = gvolt


,Open,High,Low,Close,Adj Close,Volume,Volt,Return,illq,gvolt
Date,,,,,,,,,,
2020-02-18,39.490002,39.983334,39.119999,39.876667,37.312325,34539600,0.021829,0.014760,0.004273,0.015657
2020-02-19,39.833332,39.980000,39.226665,39.226665,36.704117,21563400,0.019023,0.016300,0.007559,0.015762
2020-02-20,39.070000,39.369999,38.953335,39.230000,36.707241,15068700,0.010640,0.000085,0.000056,0.020529
2020-02-21,39.146667,39.583332,39.103333,39.526669,36.984833,18726300,0.012200,0.007562,0.004038,0.020036
2020-02-24,39.153332,39.486668,38.650002,38.773335,36.279938,19848000,0.021416,0.019059,0.009602,0.018582
...,...,...,...,...,...,...,...,...,...,...
2022-08-22,45.400002,45.573334,44.813332,44.856667,43.867420,18667200,0.016817,0.017881,0.009579,0.025569
2022-08-23,44.869999,44.986668,44.403332,44.666668,43.681614,16955100,0.013052,0.004236,0.002498,0.022652
2022-08-24,44.783333,45.096668,44.533333,44.990002,43.997814,15766500,0.012570,0.007239,0.004591,0.021196


# S&P500

In [369]:
sp500_data = yf.download('^GSPC',start='2020-01-13', end ='2023-05-05')
sp500_data['Daily_Return'] = sp500_data['Close'].pct_change()
sp500_data['Daily_Return']
sp500 = np.array([
                  sp500_data['Daily_Return'].mean(),
                  sp500_data['Daily_Return'].std(),
                  sp500_data['Daily_Return'].min(),
                  sp500_data['Daily_Return'].quantile(0.25),
                  sp500_data['Daily_Return'].quantile(0.5),
                  sp500_data['Daily_Return'].quantile(0.75),
                  sp500_data['Daily_Return'].max()
 ])
sp500
###########################

[*********************100%%**********************]  1 of 1 completed


array([ 0.00037547,  0.01558047, -0.11984055, -0.00667834,  0.00081998,
        0.00821083,  0.09382774])

In [338]:
sp500

array(['S&P 500', '0.0003754730574126004', '0.015580469071277456',
       '-0.11984055240393443', '-0.006678337608227225',
       '0.0008199802697488323', '0.008210832780053412',
       '0.09382773976227554'], dtype='<U32')

In [105]:
sp = sp500_data.loc["2020-02-15":"2022-08-27"]
sp = sp.rename(columns={"Close":"s&p500"})

sp = sp['s&p500']
sp

Date
2020-02-18    3370.290039
2020-02-19    3386.149902
2020-02-20    3373.229980
2020-02-21    3337.750000
2020-02-24    3225.889893
                 ...     
2022-08-22    4137.990234
2022-08-23    4128.729980
2022-08-24    4140.770020
2022-08-25    4199.120117
2022-08-26    4057.659912
Name: s&p500, Length: 638, dtype: float64

# COVID19


In [28]:
covid19 = pd.read_csv('covid19.csv')
covid19=covid19[covid19["location"] == "United States"]
# covid19 = covid19.iloc[365585,366792]
# covid19=covid19
covid19 = covid19.loc[(covid19["date"]>="2020-01-13")&(covid19["date"]<="2023-05-05")]


In [29]:
# covid19= covid19.set_index("date")

# covid19 = covid19.drop(columns=["level_0"])
# covid19["weekly_cases"]

In [349]:
 cases= np.array([
                  covid19['weekly_cases'].mean(),
                  covid19['weekly_cases'].std(),
                  covid19['weekly_cases'].min(),
                  covid19['weekly_cases'].quantile(0.25),
                  covid19['weekly_cases'].quantile(0.5),
                  covid19['weekly_cases'].quantile(0.75),
                  covid19['weekly_cases'].max()
 ])
deaths = np.array([
                  covid19['weekly_deaths'].mean(),
                  covid19['weekly_deaths'].std(),
                  covid19['weekly_deaths'].min(),
                  covid19['weekly_deaths'].quantile(0.25),
                  covid19['weekly_deaths'].quantile(0.5),
                  covid19['weekly_deaths'].quantile(0.75),
                  covid19['weekly_deaths'].max()
 ])
print(cases,deaths,sep="\n")
##############################

[ 597831.55004136  776321.66671525       0.          222579.
  387634.          709637.         5650933.        ]
[ 6507.3573201  5505.6123493     0.         2560.         4493.
  9065.        23312.       ]


In [16]:
index = wmt.index
ind = index
# Преобразование DatetimeIndex в массив строк без времени
array_without_time = ind.strftime('%Y-%m-%d')

# Преобразование массива строк в массив NumPy
ind = array_without_time.to_numpy()

# Вывод результата

In [52]:
cv = covid19.loc[(covid19["date"]>="2020-02-15")&(covid19["date"]<="2022-08-27")]
# cv = cv.loc[cv["date"].to_numpy().isin(index)]
ind2 = cv["date"].values
res = np.in1d(ind2,ind)
cv = cv[res]
cv = cv[["date","weekly_cases","weekly_deaths"]]
cv

,date,weekly_cases,weekly_deaths
365621,2020-02-18,4.0,0.0
365622,2020-02-19,4.0,0.0
365623,2020-02-20,4.0,0.0
365624,2020-02-21,4.0,0.0
365627,2020-02-24,16.0,0.0
...,...,...,...
366537,2022-08-22,644925.0,3231.0
366538,2022-08-23,644925.0,3231.0
366539,2022-08-24,644925.0,3231.0
366540,2022-08-25,644925.0,3231.0


,date,weekly_cases,weekly_deaths
365621,2020-02-18,4.0,0.0
365622,2020-02-19,4.0,0.0
365623,2020-02-20,4.0,0.0
365624,2020-02-21,4.0,0.0
365627,2020-02-24,16.0,0.0
...,...,...,...
366537,2022-08-22,644925.0,3231.0
366538,2022-08-23,644925.0,3231.0
366539,2022-08-24,644925.0,3231.0
366540,2022-08-25,644925.0,3231.0


# OXCGRT

In [30]:
oxcgrt = pd.read_csv("Oxcgrt.csv")


In [31]:
oxcgrt2 = oxcgrt.drop_duplicates(subset = ["Date"])


In [20]:
lol3=[]
def format_date(date):
    return '-'.join([date[:4], date[4:6], date[6:]])

lol2 = oxcgrt2["Date"].values
lol2 = lol2.astype(str)
for i in lol2:
    lol3.append(format_date(i))
lol3 = np.array(lol3)
lol3

array(['2020-01-01', '2020-01-02', '2020-01-03', ..., '2022-12-29',
       '2022-12-30', '2022-12-31'], dtype='<U10')

In [39]:
res2 = np.in1d(lol3,ind)
lol4 = lol3[res2]
oxf = oxcgrt2[res2]
oxf["Date"] = lol4
oxf = oxf[["Date","StringencyIndex_Average"]]
oxf

C:\Users\casov\AppData\Local\Temp\ipykernel_10472\617018070.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oxf["Date"] = lol4


,Date,StringencyIndex_Average
48,2020-02-18,8.33
49,2020-02-19,8.33
50,2020-02-20,8.33
51,2020-02-21,8.33
54,2020-02-24,8.33
...,...,...
964,2022-08-22,24.21
965,2022-08-23,24.21
966,2022-08-24,24.21
967,2022-08-25,24.21


In [350]:
string= np.array([
                  oxcgrt['StringencyIndex_Average'].mean(),
                  oxcgrt['StringencyIndex_Average'].std(),
                  oxcgrt['StringencyIndex_Average'].min(),
                  oxcgrt['StringencyIndex_Average'].quantile(0.25),
                  oxcgrt['StringencyIndex_Average'].quantile(0.5),
                  oxcgrt['StringencyIndex_Average'].quantile(0.75),
                  oxcgrt['StringencyIndex_Average'].max()
 ])
string
######################

array([38.95798841, 19.70473497,  0.        , 22.84      , 31.69      ,
       55.09      , 93.52      ])

# GTREND

In [351]:
gtrend = pd.read_csv("gtrend1.csv")

gtrend = gtrend['Категория: Все категории']
# gtrend.drop([1])
gtrend = np.array([gtrend])
gtrend = np.delete(gtrend,0)
gtrend[gtrend=='<1'] = 0
gtrend =gtrend.astype(float)
gtrends= np.array([
                  gtrend.mean(),
                  gtrend.std(),
                  gtrend.min(),
                  np.quantile(gtrend,0.25),
                  np.quantile(gtrend,0.5),
                  np.quantile(gtrend,0.75),
                  gtrend.max()
 ])
gtrends
#########################

array([  4.90076336,  14.65551245,   0.        ,   0.        ,
         0.        ,   2.        , 100.        ])

In [32]:
# gtrend

In [25]:
gtr = np.repeat(gtrend,7)
gtr = gtr[:-5]
data_range = pd.date_range(start = "2019-04-28",end = "2024-04-29")
df = pd.DataFrame({"gtr":gtr,"date":data_range})
array_without_time2 = data_range.strftime('%Y-%m-%d')

# Преобразование массива строк в массив NumPy
data_range2 = array_without_time2.to_numpy()
res4 = np.in1d(data_range2,ind)
gtr = df[res4]
gtr

,gtr,date
296,4.0,2020-02-18
297,4.0,2020-02-19
298,4.0,2020-02-20
299,4.0,2020-02-21
302,16.0,2020-02-24
...,...,...
1212,0.0,2022-08-22
1213,0.0,2022-08-23
1214,0.0,2022-08-24
1215,0.0,2022-08-25


# MOBILITY

In [26]:
mobility = pd.read_csv("mobl.csv")

mobility = mobility[["date","retail_and_recreation_percent_change_from_baseline",
                    "grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline",
                     "transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline",
                     "residential_percent_change_from_baseline"
                    ]]
mobility["sum"] = mobility["retail_and_recreation_percent_change_from_baseline"]+ mobility["grocery_and_pharmacy_percent_change_from_baseline"] +mobility["parks_percent_change_from_baseline"]+mobility["transit_stations_percent_change_from_baseline"] +mobility["workplaces_percent_change_from_baseline"] + mobility["residential_percent_change_from_baseline"]
mobility = mobility[["date","sum"]]
mob20 = mobility.drop_duplicates(subset = ["date"])

mobilit2 = pd.read_csv("mobl21.csv")

mobilit2 = mobilit2[["date","retail_and_recreation_percent_change_from_baseline",
                    "grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline",
                     "transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline",
                     "residential_percent_change_from_baseline"
                    ]]
mobilit2["sum"] = mobilit2["retail_and_recreation_percent_change_from_baseline"]+ mobilit2["grocery_and_pharmacy_percent_change_from_baseline"] +mobilit2["parks_percent_change_from_baseline"]+mobilit2["transit_stations_percent_change_from_baseline"] +mobilit2["workplaces_percent_change_from_baseline"] + mobilit2["residential_percent_change_from_baseline"]
mobilit2 = mobilit2[["date","sum"]]
mob21 = mobilit2.drop_duplicates(subset = ["date"])

mobilit3 = pd.read_csv("mobl22.csv")

mobilit3 = mobilit3[["date","retail_and_recreation_percent_change_from_baseline",
                    "grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline",
                     "transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline",
                     "residential_percent_change_from_baseline"
                    ]]
mobilit3["sum"] = mobilit3["retail_and_recreation_percent_change_from_baseline"]+ mobilit3["grocery_and_pharmacy_percent_change_from_baseline"] +mobilit3["parks_percent_change_from_baseline"]+mobilit3["transit_stations_percent_change_from_baseline"] +mobilit3["workplaces_percent_change_from_baseline"] + mobilit3["residential_percent_change_from_baseline"]
mobilit3 = mobilit3[["date","sum"]]
mob22 = mobilit3.drop_duplicates(subset = ["date"])


mobil = pd.concat([mob20,mob21,mob22])

ind5 = mobil['date'].values
res5 = np.in1d(ind5,ind)
mobil = mobil[res5]
mobil

,date,sum
3,2020-02-18,7.0
4,2020-02-19,12.0
5,2020-02-20,6.0
6,2020-02-21,5.0
9,2020-02-24,16.0
...,...,...
233,2022-08-22,-28.0
234,2022-08-23,-11.0
235,2022-08-24,-9.0
236,2022-08-25,-6.0


# USDWT

In [27]:
usdwt= np.array(["usdwt",
                  mobil["sum"].mean(),
                  mobil["sum"].std(),
                  mobil["sum"].min(),
                  mobil["sum"].quantile(0.25),
                  mobil["sum"].quantile(0.5),
                  mobil["sum"].quantile(0.75),
                  mobil["sum"].max()
 ])
usdwt

array(['usdwt', '-46.69749216300941', '42.8051445724009', '-183.0',
       '-73.0', '-39.0', '-15.0', '45.0'], dtype='<U32')

In [172]:
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###

# REGRESSION!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [128]:
wm = wmt[["Close","Volume","Volt","gvolt","illq"]]
wm = wm.rename(columns={"Close":"Walmart"})


In [193]:
cov = cv.set_index("date")
ox=oxf.set_index("Date")
mb = mobil.set_index("date")
gt = gtr.set_index("date")

cov.index = pd.to_datetime(cov.index)
ox.index = pd.to_datetime(ox.index)
mb.index = pd.to_datetime(mb.index)
wm.index.name = "date"
ox.index.name = "date"
sp.index.name = 'date'
data = pd.concat([wm,sp,cov,ox,mb,gt],axis=1)
data = data.rename(columns={"StringencyIndex_Average":"stringency"})
data = data.rename(columns={"sum":"mobility"})
data = data.rename(columns={"weekly_cases":"cases"})
data = data.rename(columns={"weekly_deaths":"deaths"})


data




,Walmart,Volume,Volt,gvolt,illq,s&p500,cases,deaths,stringency,mobility,gtr
date,,,,,,,,,,,
2020-02-18,39.876667,34539600,0.021829,0.015657,0.004273,3370.290039,4.0,0.0,8.33,7.0,4.0
2020-02-19,39.226665,21563400,0.019023,0.015762,0.007559,3386.149902,4.0,0.0,8.33,12.0,4.0
2020-02-20,39.230000,15068700,0.010640,0.020529,0.000056,3373.229980,4.0,0.0,8.33,6.0,4.0
2020-02-21,39.526669,18726300,0.012200,0.020036,0.004038,3337.750000,4.0,0.0,8.33,5.0,4.0
2020-02-24,38.773335,19848000,0.021416,0.018582,0.009602,3225.889893,16.0,0.0,8.33,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,44.856667,18667200,0.016817,0.025569,0.009579,4137.990234,644925.0,3231.0,24.21,-28.0,0.0
2022-08-23,44.666668,16955100,0.013052,0.022652,0.002498,4128.729980,644925.0,3231.0,24.21,-11.0,0.0
2022-08-24,44.990002,15766500,0.012570,0.021196,0.004591,4140.770020,644925.0,3231.0,24.21,-9.0,0.0


In [194]:
# data["weekly_cases"] = np.log(data["weekly_cases"])
# data
data["cases"] = np.log(data["cases"]+1)
data["deaths"] = np.log(data["deaths"] + 1)
data["stringency"] = np.log(data["stringency"] + 1)
data["gtr"] = np.log(data["gtr"] + 1)
data["illq"] = np.log(data['illq'])
data['Volt'] = np.log(data["Volt"])
data["gvolt"] = np.log(data["gvolt"])
data

,Walmart,Volume,Volt,gvolt,illq,s&p500,cases,deaths,stringency,mobility,gtr
date,,,,,,,,,,,
2020-02-18,39.876667,34539600,-3.824521,-4.156818,-5.455385,3370.290039,1.609438,0.000000,2.233235,7.0,1.609438
2020-02-19,39.226665,21563400,-3.962130,-4.150138,-4.884984,3386.149902,1.609438,0.000000,2.233235,12.0,1.609438
2020-02-20,39.230000,15068700,-4.543164,-3.885899,-9.782960,3373.229980,1.609438,0.000000,2.233235,6.0,1.609438
2020-02-21,39.526669,18726300,-4.406284,-3.910202,-5.511924,3337.750000,1.609438,0.000000,2.233235,5.0,1.609438
2020-02-24,38.773335,19848000,-3.843604,-3.985552,-4.645740,3225.889893,2.833213,0.000000,2.233235,16.0,2.833213
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,44.856667,18667200,-4.085360,-3.666391,-4.648223,4137.990234,13.376891,8.080856,3.227241,-28.0,0.000000
2022-08-23,44.666668,16955100,-4.338840,-3.787514,-5.992194,4128.729980,13.376891,8.080856,3.227241,-11.0,0.000000
2022-08-24,44.990002,15766500,-4.376410,-3.853950,-5.383600,4140.770020,13.376891,8.080856,3.227241,-9.0,0.000000


In [195]:
matr = data.cov()
matr
rank=np.linalg.matrix_rank(matr)
corr = data.corr()
print(rank)
corr

6


,Walmart,Volume,Volt,gvolt,illq,s&p500,cases,deaths,stringency,mobility,gtr
Walmart,1.000000,-0.237249,-0.393753,-0.454589,-0.066310,0.647000,0.487163,0.464711,0.147935,-0.127355,-0.473410
Volume,-0.237249,1.000000,0.602284,0.310568,0.089837,-0.173383,-0.129146,-0.128645,-0.015176,-0.020337,0.148794
Volt,-0.393753,0.602284,1.000000,0.560687,0.344260,-0.406872,-0.286749,-0.310315,-0.171848,-0.081673,0.314603
gvolt,-0.454589,0.310568,0.560687,1.000000,0.099895,-0.476738,-0.328250,-0.372512,-0.101622,-0.111118,0.380424
illq,-0.066310,0.089837,0.344260,0.099895,1.000000,-0.065010,-0.104222,-0.111095,-0.114194,0.008958,0.023076
s&p500,0.647000,-0.173383,-0.406872,-0.476738,-0.065010,1.000000,0.443979,0.282086,-0.264066,0.258796,-0.856876
cases,0.487163,-0.129146,-0.286749,-0.328250,-0.104222,0.443979,1.000000,0.900142,0.460472,-0.309953,-0.417629
deaths,0.464711,-0.128645,-0.310315,-0.372512,-0.111095,0.282086,0.900142,1.000000,0.670625,-0.476122,-0.171671
stringency,0.147935,-0.015176,-0.171848,-0.101622,-0.114194,-0.264066,0.460472,0.670625,1.000000,-0.532052,0.446519
mobility,-0.127355,-0.020337,-0.081673,-0.111118,0.008958,0.258796,-0.309953,-0.476122,-0.532052,1.000000,-0.367786


In [186]:
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

# WALMART VOL

In [242]:
def significance_stars(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

data['constant'] = 1
model1 = sm.OLS(data['Volt'], data[['constant', 'deaths', 'Walmart', 's&p500', 'illq']])
result1 = model1.fit(cov_type='HC1')  # Для robust standard errors
results_summary1 = result1.summary()
p_values1 = result1.pvalues
table_data1 = {
    'b_deaths': [result1.params['deaths'], significance_stars(p_values1['deaths'])],
    'b_gtr':[np.nan],
    'b_mobility':[np.nan],
    'b_stringency':[np.nan],
    'b_cases':[np.nan],
    'b_illq': [result1.params['illq'], significance_stars(p_values1['illq'])],
    'b_Walmart': [result1.params['Walmart'], significance_stars(p_values1['Walmart'])],
    'b_s&p500': [result1.params['s&p500'], significance_stars(p_values1['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result1.bse,
    'observations': result1.nobs,
    'r-squared': result1.rsquared
}
vol1 = pd.DataFrame(table_data1, index=['vol1', '*'])

model2 = sm.OLS(data['Volt'], data[['constant', 'gtr', 'Walmart', 's&p500', 'illq']])
result2 = model2.fit(cov_type='HC1')  # Для robust standard errors
results_summary2 = result2.summary()
p_values2 = result2.pvalues
table_data2 = {
    'b_gtr': [result2.params['gtr'], significance_stars(p_values2['gtr'])],
    'b_illq': [result2.params['illq'], significance_stars(p_values2['illq'])],
    'b_Walmart': [result2.params['Walmart'], significance_stars(p_values2['Walmart'])],
    'b_s&p500': [result2.params['s&p500'], significance_stars(p_values2['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result2.bse,
    'observations': result2.nobs,
    'r-squared': result2.rsquared
}
vol2 = pd.DataFrame(table_data2, index=['vol2', '*'])

model3 = sm.OLS(data['Volt'], data[['constant', 'mobility', 'Walmart', 's&p500', 'illq']])
result3 = model3.fit(cov_type='HC1')  # Для robust standard errors
results_summary3 = result3.summary()
p_values3 = result3.pvalues
table_data3 = {
    'b_mobility': [result3.params['mobility'], significance_stars(p_values3['mobility'])],
    'b_illq': [result3.params['illq'], significance_stars(p_values3['illq'])],
    'b_Walmart': [result3.params['Walmart'], significance_stars(p_values3['Walmart'])],
    'b_s&p500': [result3.params['s&p500'], significance_stars(p_values3['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result3.bse,
    'observations': result3.nobs,
    'r-squared': result3.rsquared
}
vol3 = pd.DataFrame(table_data3, index=['vol3', '*'])


model4 = sm.OLS(data['Volt'], data[['constant', 'stringency', 'Walmart', 's&p500', 'illq']])
result4 = model4.fit(cov_type='HC1')  # Для robust standard errors
results_summary4 = result4.summary()
p_values4 = result4.pvalues
table_data4 = {
    'b_stringency': [result4.params['stringency'], significance_stars(p_values4['stringency'])],
    'b_illq': [result4.params['illq'], significance_stars(p_values4['illq'])],
    'b_Walmart': [result4.params['Walmart'], significance_stars(p_values4['Walmart'])],
    'b_s&p500': [result4.params['s&p500'], significance_stars(p_values4['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result4.bse,
    'observations': result4.nobs,
    'r-squared': result4.rsquared
}
vol4 = pd.DataFrame(table_data4, index=['vol4', '*'])

model5 = sm.OLS(data['Volt'], data[['constant', 'cases', 'Walmart', 's&p500', 'illq']])
result5 = model5.fit(cov_type='HC1')  # Для robust standard errors
results_summary5 = result.summary()
p_values5 = result5.pvalues
table_data5 = {
    'b_cases': [result5.params['cases'], significance_stars(p_values5['cases'])],
    'b_illq': [result5.params['illq'], significance_stars(p_values5['illq'])],
    'b_Walmart': [result5.params['Walmart'], significance_stars(p_values5['Walmart'])],
    'b_s&p500': [result5.params['s&p500'], significance_stars(p_values5['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result5.bse,
    'observations': result5.nobs,
    'r-squared': result5.rsquared
}
vol5 = pd.DataFrame(table_data5, index=['vol5', '*'])
ster = np.array([result1.scale,result2.scale,result3.scale,result4.scale,result5.scale])
fstat = np.array([result1.fvalue,result2.fvalue,result3.fvalue,result4.fvalue,result5.fvalue])
print("standart error: ", ster)
print("f statistica: ", fstat)
datf = pd.concat([vol1,vol2,vol3,vol4,vol5])
datf = datf.T
datf

standart error:  [0.17443552 0.17808783 0.17742808 0.16823661 0.17757441]
f statistica:  [50.17346732 47.77026011 46.3542972  49.55482348 46.60454329]


,vol1,*,vol2,*,vol3,*,vol4,*,vol5,*
b_deaths,-0.04335,**,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b_gtr,NaN,NaN,-0.011057,,NaN,NaN,NaN,NaN,NaN,NaN
b_mobility,NaN,NaN,NaN,NaN,-0.000698,,NaN,NaN,NaN,NaN
b_stringency,NaN,NaN,NaN,NaN,NaN,NaN,-0.295814,***,NaN,NaN
b_cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014535,
b_illq,0.12787,***,0.132268,***,0.132936,***,0.121059,***,0.131027,***
b_Walmart,-0.020971,**,-0.030018,***,-0.034914,***,-0.012472,,-0.027829,***
b_s&p500,-0.000218,***,-0.000235,***,-0.000184,***,-0.000337,***,-0.000204,***
constant,-1.151018,***,-1.151018,***,-1.151018,***,-1.151018,***,-1.151018,***
robust_se,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# WALMART GVOLT 

In [244]:
model1 = sm.OLS(data['gvolt'], data[['constant', 'deaths', 'Walmart', 's&p500', 'illq']])
result1 = model1.fit(cov_type='HC1')  # Для robust standard errors
results_summary1 = result1.summary()
p_values1 = result1.pvalues
table_data1 = {
    'b_deaths': [result1.params['deaths'], significance_stars(p_values1['deaths'])],
    'b_gtr':[np.nan],
    'b_mobility':[np.nan],
    'b_stringency':[np.nan],
    'b_cases':[np.nan],
    'b_illq': [result1.params['illq'], significance_stars(p_values1['illq'])],
    'b_Walmart': [result1.params['Walmart'], significance_stars(p_values1['Walmart'])],
    'b_s&p500': [result1.params['s&p500'], significance_stars(p_values1['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result1.bse,
    'observations': result1.nobs,
    'r-squared': result1.rsquared
}
gvol1 = pd.DataFrame(table_data1, index=['gvol1', '*'])

model2 = sm.OLS(data['gvolt'], data[['constant', 'gtr', 'Walmart', 's&p500', 'illq']])
result2 = model2.fit(cov_type='HC1')  # Для robust standard errors
results_summary2 = result2.summary()
p_values2 = result2.pvalues
table_data2 = {
    'b_gtr': [result2.params['gtr'], significance_stars(p_values2['gtr'])],
    'b_illq': [result2.params['illq'], significance_stars(p_values2['illq'])],
    'b_Walmart': [result2.params['Walmart'], significance_stars(p_values2['Walmart'])],
    'b_s&p500': [result2.params['s&p500'], significance_stars(p_values2['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result2.bse,
    'observations': result2.nobs,
    'r-squared': result2.rsquared
}
gvol2 = pd.DataFrame(table_data2, index=['gvol2', '*'])

model3 = sm.OLS(data['gvolt'], data[['constant', 'mobility', 'Walmart', 's&p500', 'illq']])
result3 = model3.fit(cov_type='HC1')  # Для robust standard errors
results_summary3 = result3.summary()
p_values3 = result3.pvalues
table_data3 = {
    'b_mobility': [result3.params['mobility'], significance_stars(p_values3['mobility'])],
    'b_illq': [result3.params['illq'], significance_stars(p_values3['illq'])],
    'b_Walmart': [result3.params['Walmart'], significance_stars(p_values3['Walmart'])],
    'b_s&p500': [result3.params['s&p500'], significance_stars(p_values3['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result3.bse,
    'observations': result3.nobs,
    'r-squared': result3.rsquared
}
gvol3 = pd.DataFrame(table_data3, index=['gvol3', '*'])


model4 = sm.OLS(data['gvolt'], data[['constant', 'stringency', 'Walmart', 's&p500', 'illq']])
result4 = model4.fit(cov_type='HC1')  # Для robust standard errors
results_summary4 = result4.summary()
p_values4 = result4.pvalues
table_data4 = {
    'b_stringency': [result4.params['stringency'], significance_stars(p_values4['stringency'])],
    'b_illq': [result4.params['illq'], significance_stars(p_values4['illq'])],
    'b_Walmart': [result4.params['Walmart'], significance_stars(p_values4['Walmart'])],
    'b_s&p500': [result4.params['s&p500'], significance_stars(p_values4['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result4.bse,
    'observations': result4.nobs,
    'r-squared': result4.rsquared
}
gvol4 = pd.DataFrame(table_data4, index=['gvol4', '*'])

model5 = sm.OLS(data['gvolt'], data[['constant', 'cases', 'Walmart', 's&p500', 'illq']])
result5 = model5.fit(cov_type='HC1')  # Для robust standard errors
results_summary5 = result.summary()
p_values5 = result5.pvalues
table_data5 = {
    'b_cases': [result5.params['cases'], significance_stars(p_values5['cases'])],
    'b_illq': [result5.params['illq'], significance_stars(p_values5['illq'])],
    'b_Walmart': [result5.params['Walmart'], significance_stars(p_values5['Walmart'])],
    'b_s&p500': [result5.params['s&p500'], significance_stars(p_values5['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result5.bse,
    'observations': result5.nobs,
    'r-squared': result5.rsquared
}
gvol5 = pd.DataFrame(table_data5, index=['gvol5', '*'])
ster = np.array([result1.scale,result2.scale,result3.scale,result4.scale,result5.scale])
fstat = np.array([result1.fvalue,result2.fvalue,result3.fvalue,result4.fvalue,result5.fvalue])
print("standart error: ", ster)
print("f statistica: ", fstat)
datf2 = pd.concat([gvol1,gvol2,gvol3,gvol4,gvol5])
datf2 = datf2.T
datf2

standart error:  [0.08000435 0.08385867 0.0832994  0.08084862 0.08326854]
f statistica:  [29.99470644 34.03335086 28.14608481 25.89240902 27.35698407]


,gvol1,*,gvol2,*,gvol3,*,gvol4,*,gvol5,*
b_deaths,-0.044355,**,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b_gtr,NaN,NaN,-0.00657,,NaN,NaN,NaN,NaN,NaN,NaN
b_mobility,NaN,NaN,NaN,NaN,-0.000631,,NaN,NaN,NaN,NaN
b_stringency,NaN,NaN,NaN,NaN,NaN,NaN,-0.163573,**,NaN,NaN
b_cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.015149,
b_illq,0.013146,,0.017786,,0.018289,,0.011601,,0.016347,
b_Walmart,-0.014444,**,-0.023926,***,-0.028188,***,-0.014245,**,-0.021409,***
b_s&p500,-0.000184,***,-0.000193,***,-0.000153,***,-0.000248,***,-0.000169,***
constant,-2.089177,***,-2.089177,***,-2.089177,***,-2.089177,***,-2.089177,***
robust_se,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:

# Добавляем столбец с константой
data['constant'] = 1

# Оцениваем модель OLS с робастными стандартными ошибками (HC1)
model = sm.OLS(data['Volt'], data[['constant', 'cases', 'Walmart', 's&p500', 'illq']])
result = model.fit(cov_type='HC1')

# Получаем робастные стандартные ошибки
robust_standard_errors = result.HC1_se

# Выводим робастные стандартные ошибки
print("Робастные стандартные ошибки:")
print(robust_standard_errors)
print()
print(v8)
print("F-stat", result.fvalue)
print("st error", result.scale)

Робастные стандартные ошибки:
constant    0.259464
cases       0.012167
Walmart     0.006417
s&p500      0.000038
illq        0.017716
dtype: float64

              constant   b_cases    b_illq b_Walmart  b_s&p500  robust_se  \
vol8         -1.056784 -0.014535  0.131027 -0.027829 -0.000204        NaN   
significance       ***                 ***       ***       ***        NaN   

              observations  r-squared  
vol8                 638.0    0.29514  
significance         638.0    0.29514  
F-stat 46.604543288715476
st error 0.1775744072746668


# Zoom

In [245]:
zm_dtf = yf.download('ZM',start = '2020-02-15',end = "2022-08-27")
zm_dtf = 

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-18,93.019997,98.139999,92.400002,96.389999,96.389999,5608200
2020-02-19,100.320000,105.080002,99.750000,103.930000,103.930000,7793800
2020-02-20,104.000000,110.750000,101.430000,105.290001,105.290001,11293200
2020-02-21,104.029999,104.290001,98.089996,101.760002,101.760002,5519200
2020-02-24,97.800003,110.000000,97.019997,105.010002,105.010002,12172800
...,...,...,...,...,...,...
2022-08-22,98.199997,100.139999,96.480003,97.440002,97.440002,8274100
2022-08-23,84.559998,87.589996,81.129997,81.320000,81.320000,31475400
2022-08-24,81.639999,85.070000,80.500000,83.610001,83.610001,11133300


In [255]:
zm_df = zm_dtf
zm_df["Volt"] = np.log(zm_df["High"]/zm_df["Low"])
zm_df["Return"] = zm_df["Close"].pct_change().abs() 
zm_df["illq"] = zm_df["Return"]/zm_df["Volume"]*10**7
zm_df['spread'] = zm_df["High"] - zm_df["Low"]
daily_returns = zm_df['Close'].pct_change()
first_diff_returns = daily_returns.diff().dropna()
garch_model = arch_model(first_diff_returns, vol='GARCH', p=1, q=1)
results = garch_model.fit()
conditional_volatility = results.conditional_volatility
gvolt  = conditional_volatility.loc["2020-02-15":"2022-08-27"]
zm_df = zm_df.loc["2020-02-15":"2022-08-27"]
zm_df["gvolt"] = gvolt
zm = zm_df[["Close","Volume","Volt","gvolt","illq"]]
zm = zm.rename(columns={"Close":"Zoom"})
data2 = pd.concat([zm,sp,cov,ox,mb,gt],axis=1)
data2 = data2.rename(columns={"StringencyIndex_Average":"stringency"})
data2 = data2.rename(columns={"sum":"mobility"})
data2 = data2.rename(columns={"weekly_cases":"cases"})
data2 = data2.rename(columns={"weekly_deaths":"deaths"})
data2["cases"] = np.log(data2["cases"]+1)
data2["deaths"] = np.log(data2["deaths"] + 1)
data2["stringency"] = np.log(data2["stringency"] + 1)
data2["gtr"] = np.log(data2["gtr"] + 1)
data2["illq"] = np.log(data2['illq'])
data2['Volt'] = np.log(data2["Volt"])
data2["gvolt"] = np.log(data2["gvolt"])
data2 = data2.loc["2020-02-20":]
data2


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1848628.0141847986
Iteration:      2,   Func. Count:     17,   Neg. LLF: 229.31633574094897
Iteration:      3,   Func. Count:     25,   Neg. LLF: 51346.51876960988
Iteration:      4,   Func. Count:     31,   Neg. LLF: -650.6559184516129
Iteration:      5,   Func. Count:     37,   Neg. LLF: -816.3591887588693
Iteration:      6,   Func. Count:     43,   Neg. LLF: -922.4044017219067
Iteration:      7,   Func. Count:     49,   Neg. LLF: -924.9792572209465
Iteration:      8,   Func. Count:     54,   Neg. LLF: -537.8790648344046
Iteration:      9,   Func. Count:     60,   Neg. LLF: 927.959864625793
Iteration:     10,   Func. Count:     67,   Neg. LLF: -920.3934141330062
Iteration:     11,   Func. Count:     73,   Neg. LLF: -917.1519324692733
Iteration:     12,   Func. Count:     79,   Neg. LLF: -926.6277860326916
Iteration:     13,   Func. Count:     84,   Neg. LLF: -926.6360515947005
Iteration:     14,   Func. Count:     89,   Neg. LLF: 

C:\Users\casov\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.004052. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\casov\AppData\Local\Temp\ipykernel_10472\1816973727.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zm_df["gvolt"] = gvolt


,Zoom,Volume,Volt,gvolt,illq,s&p500,cases,deaths,stringency,mobility,gtr
2020-02-20,105.290001,11293200,-2.431482,-2.506994,-4.457848,3373.229980,1.609438,0.000000,2.233235,6.0,1.609438
2020-02-21,101.760002,5519200,-2.792137,-2.707036,-2.801069,3337.750000,1.609438,0.000000,2.233235,5.0,1.609438
2020-02-24,105.010002,12172800,-2.074946,-2.876796,-3.640581,3225.889893,2.833213,0.000000,2.233235,16.0,2.833213
2020-02-25,107.080002,13425900,-2.690457,-2.717288,-4.221108,3128.209961,2.833213,0.000000,2.233235,18.0,2.833213
2020-02-26,106.879997,6867500,-2.945592,-3.095264,-5.907206,3116.389893,2.833213,0.000000,2.233235,18.0,2.833213
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,97.440002,8274100,-3.290548,-2.899993,-3.687998,4137.990234,13.376891,8.080856,3.227241,-28.0,0.000000
2022-08-23,81.320000,31475400,-2.568975,-3.099561,-2.945797,4128.729980,13.376891,8.080856,3.227241,-11.0,0.000000
2022-08-24,83.610001,11133300,-2.896480,-2.150779,-3.677195,4140.770020,13.376891,8.080856,3.227241,-9.0,0.000000
2022-08-25,85.720001,5409700,-3.228844,-1.869785,-3.065083,4199.120117,13.376891,8.080856,3.227241,-6.0,0.000000


# VOL


In [259]:
def significance_stars(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

data2['constant'] = 1
model1 = sm.OLS(data2['Volt'], data2[['constant', 'deaths', 'Zoom', 's&p500', 'illq']])
result1 = model1.fit(cov_type='HC1')  # Для robust standard errors
results_summary1 = result1.summary()
p_values1 = result1.pvalues
table_data1 = {
    'b_deaths': [result1.params['deaths'], significance_stars(p_values1['deaths'])],
    'b_gtr':[np.nan],
    'b_mobility':[np.nan],
    'b_stringency':[np.nan],
    'b_cases':[np.nan],
    'b_illq': [result1.params['illq'], significance_stars(p_values1['illq'])],
    'b_Zoom': [result1.params['Zoom'], significance_stars(p_values1['Zoom'])],
    'b_s&p500': [result1.params['s&p500'], significance_stars(p_values1['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result1.bse,
    'observations': result1.nobs,
    'r-squared': result1.rsquared
}
vol1 = pd.DataFrame(table_data1, index=['vol1', '*'])

model2 = sm.OLS(data2['Volt'], data2[['constant', 'gtr', 'Zoom', 's&p500', 'illq']])
result2 = model2.fit(cov_type='HC1')  # Для robust standard errors
results_summary2 = result2.summary()
p_values2 = result2.pvalues
table_data2 = {
    'b_gtr': [result2.params['gtr'], significance_stars(p_values2['gtr'])],
    'b_illq': [result2.params['illq'], significance_stars(p_values2['illq'])],
    'b_Zoom': [result2.params['Zoom'], significance_stars(p_values2['Zoom'])],
    'b_s&p500': [result2.params['s&p500'], significance_stars(p_values2['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result2.bse,
    'observations': result2.nobs,
    'r-squared': result2.rsquared
}
vol2 = pd.DataFrame(table_data2, index=['vol2', '*'])

model3 = sm.OLS(data2['Volt'], data2[['constant', 'mobility', 'Zoom', 's&p500', 'illq']])
result3 = model3.fit(cov_type='HC1')  # Для robust standard errors
results_summary3 = result3.summary()
p_values3 = result3.pvalues
table_data3 = {
    'b_mobility': [result3.params['mobility'], significance_stars(p_values3['mobility'])],
    'b_illq': [result3.params['illq'], significance_stars(p_values3['illq'])],
    'b_Zoom': [result3.params['Zoom'], significance_stars(p_values3['Zoom'])],
    'b_s&p500': [result3.params['s&p500'], significance_stars(p_values3['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result3.bse,
    'observations': result3.nobs,
    'r-squared': result3.rsquared
}
vol3 = pd.DataFrame(table_data3, index=['vol3', '*'])


model4 = sm.OLS(data2['Volt'], data2[['constant', 'stringency', 'Zoom', 's&p500', 'illq']])
result4 = model4.fit(cov_type='HC1')  # Для robust standard errors
results_summary4 = result4.summary()
p_values4 = result4.pvalues
table_data4 = {
    'b_stringency': [result4.params['stringency'], significance_stars(p_values4['stringency'])],
    'b_illq': [result4.params['illq'], significance_stars(p_values4['illq'])],
    'b_Zoom': [result4.params['Zoom'], significance_stars(p_values4['Zoom'])],
    'b_s&p500': [result4.params['s&p500'], significance_stars(p_values4['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result4.bse,
    'observations': result4.nobs,
    'r-squared': result4.rsquared
}
vol4 = pd.DataFrame(table_data4, index=['vol4', '*'])

model5 = sm.OLS(data2['Volt'], data2[['constant', 'cases', 'Zoom', 's&p500', 'illq']])
result5 = model5.fit(cov_type='HC1')  # Для robust standard errors
results_summary5 = result.summary()
p_values5 = result5.pvalues
table_data5 = {
    'b_cases': [result5.params['cases'], significance_stars(p_values5['cases'])],
    'b_illq': [result5.params['illq'], significance_stars(p_values5['illq'])],
    'b_Zoom': [result5.params['Zoom'], significance_stars(p_values5['Zoom'])],
    'b_s&p500': [result5.params['s&p500'], significance_stars(p_values5['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result5.bse,
    'observations': result5.nobs,
    'r-squared': result5.rsquared
}
vol5 = pd.DataFrame(table_data5, index=['vol5', '*'])
ster = np.array([result1.scale,result2.scale,result3.scale,result4.scale,result5.scale])
fstat = np.array([result1.fvalue,result2.fvalue,result3.fvalue,result4.fvalue,result5.fvalue])
print("standart error: ", ster)
print("f statistica: ", fstat)
datf = pd.concat([vol1,vol2,vol3,vol4,vol5])
datf = datf.T
datf

standart error:  [0.15435502 0.15444322 0.15161975 0.15148901 0.15522439]
f statistica:  [64.54202761 65.14219804 65.80590813 68.2300927  61.9306023 ]


,vol1,*,vol2,*,vol3,*,vol4,*,vol5,*
b_deaths,-0.025805,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b_gtr,NaN,NaN,-0.057848,*,NaN,NaN,NaN,NaN,NaN,NaN
b_mobility,NaN,NaN,NaN,NaN,-0.001562,***,NaN,NaN,NaN,NaN
b_stringency,NaN,NaN,NaN,NaN,NaN,NaN,-0.231121,***,NaN,NaN
b_cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.013097,
b_illq,0.094579,***,0.089752,***,0.09149,***,0.09362,***,0.094889,***
b_Zoom,-0.000921,***,-0.000968,***,-0.001117,***,-0.000574,***,-0.000977,***
b_s&p500,-0.000321,***,-0.00044,***,-0.00031,***,-0.00038,***,-0.000323,***
constant,-1.009208,***,-1.009208,***,-1.009208,***,-1.009208,***,-1.009208,***
robust_se,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ZOOM GVOLT

In [260]:
model1 = sm.OLS(data2['gvolt'], data2[['constant', 'deaths', 'Zoom', 's&p500', 'illq']])
result1 = model1.fit(cov_type='HC1')  # Для robust standard errors
results_summary1 = result1.summary()
p_values1 = result1.pvalues
table_data1 = {
    'b_deaths': [result1.params['deaths'], significance_stars(p_values1['deaths'])],
    'b_gtr':[np.nan],
    'b_mobility':[np.nan],
    'b_stringency':[np.nan],
    'b_cases':[np.nan],
    'b_illq': [result1.params['illq'], significance_stars(p_values1['illq'])],
    'b_Zoom': [result1.params['Zoom'], significance_stars(p_values1['Zoom'])],
    'b_s&p500': [result1.params['s&p500'], significance_stars(p_values1['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result1.bse,
    'observations': result1.nobs,
    'r-squared': result1.rsquared
}
gvol1 = pd.DataFrame(table_data1, index=['gvol1', '*'])

model2 = sm.OLS(data2['gvolt'], data2[['constant', 'gtr', 'Zoom', 's&p500', 'illq']])
result2 = model2.fit(cov_type='HC1')  # Для robust standard errors
results_summary2 = result2.summary()
p_values2 = result2.pvalues
table_data2 = {
    'b_gtr': [result2.params['gtr'], significance_stars(p_values2['gtr'])],
    'b_illq': [result2.params['illq'], significance_stars(p_values2['illq'])],
    'b_Zoom': [result2.params['Zoom'], significance_stars(p_values2['Zoom'])],
    'b_s&p500': [result2.params['s&p500'], significance_stars(p_values2['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result2.bse,
    'observations': result2.nobs,
    'r-squared': result2.rsquared
}
gvol2 = pd.DataFrame(table_data2, index=['gvol2', '*'])

model3 = sm.OLS(data2['gvolt'], data2[['constant', 'mobility', 'Zoom', 's&p500', 'illq']])
result3 = model3.fit(cov_type='HC1')  # Для robust standard errors
results_summary3 = result3.summary()
p_values3 = result3.pvalues
table_data3 = {
    'b_mobility': [result3.params['mobility'], significance_stars(p_values3['mobility'])],
    'b_illq': [result3.params['illq'], significance_stars(p_values3['illq'])],
    'b_Zoom': [result3.params['Zoom'], significance_stars(p_values3['Zoom'])],
    'b_s&p500': [result3.params['s&p500'], significance_stars(p_values3['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result3.bse,
    'observations': result3.nobs,
    'r-squared': result3.rsquared
}
gvol3 = pd.DataFrame(table_data3, index=['gvol3', '*'])


model4 = sm.OLS(data2['gvolt'], data2[['constant', 'stringency', 'Zoom', 's&p500', 'illq']])
result4 = model4.fit(cov_type='HC1')  # Для robust standard errors
results_summary4 = result4.summary()
p_values4 = result4.pvalues
table_data4 = {
    'b_stringency': [result4.params['stringency'], significance_stars(p_values4['stringency'])],
    'b_illq': [result4.params['illq'], significance_stars(p_values4['illq'])],
    'b_Zoom': [result4.params['Zoom'], significance_stars(p_values4['Zoom'])],
    'b_s&p500': [result4.params['s&p500'], significance_stars(p_values4['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result4.bse,
    'observations': result4.nobs,
    'r-squared': result4.rsquared
}
gvol4 = pd.DataFrame(table_data4, index=['gvol4', '*'])

model5 = sm.OLS(data2['gvolt'], data2[['constant', 'cases', 'Zoom', 's&p500', 'illq']])
result5 = model5.fit(cov_type='HC1')  # Для robust standard errors
results_summary5 = result.summary()
p_values5 = result5.pvalues
table_data5 = {
    'b_cases': [result5.params['cases'], significance_stars(p_values5['cases'])],
    'b_illq': [result5.params['illq'], significance_stars(p_values5['illq'])],
    'b_Zoom': [result5.params['Zoom'], significance_stars(p_values5['Zoom'])],
    'b_s&p500': [result5.params['s&p500'], significance_stars(p_values5['s&p500'])],
    'constant': [result1.params['constant'], significance_stars(p_values1['constant'])],
    'robust_se': result5.bse,
    'observations': result5.nobs,
    'r-squared': result5.rsquared
}
gvol5 = pd.DataFrame(table_data5, index=['gvol5', '*'])
ster = np.array([result1.scale,result2.scale,result3.scale,result4.scale,result5.scale])
fstat = np.array([result1.fvalue,result2.fvalue,result3.fvalue,result4.fvalue,result5.fvalue])
print("standart error: ", ster)
print("f statistica: ", fstat)
datf2 = pd.concat([gvol1,gvol2,gvol3,gvol4,gvol5])
datf2 = datf2.T
datf2

standart error:  [0.0860367  0.08560307 0.08562947 0.08554171 0.08605018]
f statistica:  [ 7.37168111 10.29871744  7.40116221  8.61805625  7.35580442]


,gvol1,*,gvol2,*,gvol3,*,gvol4,*,gvol5,*
b_deaths,-0.002712,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b_gtr,NaN,NaN,-0.03533,,NaN,NaN,NaN,NaN,NaN,NaN
b_mobility,NaN,NaN,NaN,NaN,-0.000505,,NaN,NaN,NaN,NaN
b_stringency,NaN,NaN,NaN,NaN,NaN,NaN,-0.080912,,NaN,NaN
b_cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000555,
b_illq,-0.006401,,-0.00914,,-0.00731,,-0.006635,,-0.006475,
b_Zoom,-0.00031,**,-0.000291,**,-0.000353,***,-0.000166,,-0.000321,**
b_s&p500,-0.0001,***,-0.000162,***,-0.000092,***,-0.000116,***,-0.000103,***
constant,-2.406411,***,-2.406411,***,-2.406411,***,-2.406411,***,-2.406411,***
robust_se,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
sp

date
2020-02-18    3370.290039
2020-02-19    3386.149902
2020-02-20    3373.229980
2020-02-21    3337.750000
2020-02-24    3225.889893
                 ...     
2022-08-22    4137.990234
2022-08-23    4128.729980
2022-08-24    4140.770020
2022-08-25    4199.120117
2022-08-26    4057.659912
Name: s&p500, Length: 638, dtype: float64

In [316]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(sp, labels={'ts': 'Время', 'prices': 'Цена'}, title='S&P 500',template = 'plotly_dark')
fig.update_traces(line=dict(color='cyan', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='black', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red',range=[0,8000])  # Добавить сетку на оси y
fig.show()





In [268]:
data

,Walmart,Volume,Volt,gvolt,illq,s&p500,cases,deaths,stringency,mobility,gtr,constant
date,,,,,,,,,,,,
2020-02-18,39.876667,34539600,-3.824521,-4.156818,-5.455385,3370.290039,1.609438,0.000000,2.233235,7.0,1.609438,1
2020-02-19,39.226665,21563400,-3.962130,-4.150138,-4.884984,3386.149902,1.609438,0.000000,2.233235,12.0,1.609438,1
2020-02-20,39.230000,15068700,-4.543164,-3.885899,-9.782960,3373.229980,1.609438,0.000000,2.233235,6.0,1.609438,1
2020-02-21,39.526669,18726300,-4.406284,-3.910202,-5.511924,3337.750000,1.609438,0.000000,2.233235,5.0,1.609438,1
2020-02-24,38.773335,19848000,-3.843604,-3.985552,-4.645740,3225.889893,2.833213,0.000000,2.233235,16.0,2.833213,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,44.856667,18667200,-4.085360,-3.666391,-4.648223,4137.990234,13.376891,8.080856,3.227241,-28.0,0.000000,1
2022-08-23,44.666668,16955100,-4.338840,-3.787514,-5.992194,4128.729980,13.376891,8.080856,3.227241,-11.0,0.000000,1
2022-08-24,44.990002,15766500,-4.376410,-3.853950,-5.383600,4140.770020,13.376891,8.080856,3.227241,-9.0,0.000000,1


In [313]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(data['Walmart'], labels={'ts': 'Время', 'prices': 'Цена'}, title='Walmart Inc.')
fig.update_traces(line=dict(color='red', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='pink', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red',range=[0,100])  # Добавить сетку на оси y
fig.show()





In [311]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(data2['Zoom'], labels={'ts': 'Время', 'prices': 'Цена'}, title='Zoom Video Communications',template = "plotly_dark")
fig.update_traces(line=dict(color='cyan', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='black', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси y
fig.show()





In [293]:
data

,Walmart,Volume,Volt,gvolt,illq,s&p500,cases,deaths,stringency,mobility,gtr,constant
date,,,,,,,,,,,,
2020-02-18,39.876667,34539600,-3.824521,-4.156818,-5.455385,3370.290039,1.609438,0.000000,2.233235,7.0,1.609438,1
2020-02-19,39.226665,21563400,-3.962130,-4.150138,-4.884984,3386.149902,1.609438,0.000000,2.233235,12.0,1.609438,1
2020-02-20,39.230000,15068700,-4.543164,-3.885899,-9.782960,3373.229980,1.609438,0.000000,2.233235,6.0,1.609438,1
2020-02-21,39.526669,18726300,-4.406284,-3.910202,-5.511924,3337.750000,1.609438,0.000000,2.233235,5.0,1.609438,1
2020-02-24,38.773335,19848000,-3.843604,-3.985552,-4.645740,3225.889893,2.833213,0.000000,2.233235,16.0,2.833213,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,44.856667,18667200,-4.085360,-3.666391,-4.648223,4137.990234,13.376891,8.080856,3.227241,-28.0,0.000000,1
2022-08-23,44.666668,16955100,-4.338840,-3.787514,-5.992194,4128.729980,13.376891,8.080856,3.227241,-11.0,0.000000,1
2022-08-24,44.990002,15766500,-4.376410,-3.853950,-5.383600,4140.770020,13.376891,8.080856,3.227241,-9.0,0.000000,1


In [301]:
# Вычисление матрицы ковариации
pic1 = data.drop("constant",axis=1)
pic1.columns = x=['Walmart',"Volume","Volatility","GVolatility","Illiquidity","S&P 500","Cases","Deaths","Stringency","Mobility","Google Trends"]
cov_matrix = pic1.corr()

# Построение тепловой карты с настроенной цветовой палитрой
fig = px.imshow(cov_matrix, color_continuous_scale='Viridis',title="Матрица корреляции для компании Walmart Inc.")
fig.update_layout(width=650,height=650)
for i in range(len(cov_matrix)):
    for j in range(len(cov_matrix.columns)):
        fig.add_annotation(text=str(cov_matrix.iloc[i, j].round(2)),
                           x=cov_matrix.columns[j], y=cov_matrix.index[i],
                           showarrow=False, font=dict(color='black'))
fig.show()

In [304]:
# Вычисление матрицы ковариации
pic2 = data2.drop("constant",axis=1)
pic2.columns = x=['Zoom',"Volume","Volatility","GVolatility","Illiquidity","S&P 500","Cases","Deaths","Stringency","Mobility","Google Trends"]
cov_matrix = pic2.corr()

# Построение тепловой карты с настроенной цветовой палитрой
fig = px.imshow(cov_matrix, color_continuous_scale='Blues',title="Матрица корреляции для компании Zoom Video Communications")
fig.update_layout(width=650,height=650)
for i in range(len(cov_matrix)):
    for j in range(len(cov_matrix.columns)):
        fig.add_annotation(text=str(cov_matrix.iloc[i, j].round(2)),
                           x=cov_matrix.columns[j], y=cov_matrix.index[i],
                           showarrow=False, font=dict(color='black'))
fig.show()

In [305]:
mobil

,date,sum
3,2020-02-18,7.0
4,2020-02-19,12.0
5,2020-02-20,6.0
6,2020-02-21,5.0
9,2020-02-24,16.0
...,...,...
233,2022-08-22,-28.0
234,2022-08-23,-11.0
235,2022-08-24,-9.0
236,2022-08-25,-6.0


In [323]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(mobil,x='date', y='sum',labels={'ts': 'Время', 'prices': 'Цена'}, title='Mobility',template = 'plotly_dark')
fig.update_traces(line=dict(color='cyan', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='black', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси y
fig.show()





In [324]:
cv

,date,weekly_cases,weekly_deaths
365621,2020-02-18,4.0,0.0
365622,2020-02-19,4.0,0.0
365623,2020-02-20,4.0,0.0
365624,2020-02-21,4.0,0.0
365627,2020-02-24,16.0,0.0
...,...,...,...
366537,2022-08-22,644925.0,3231.0
366538,2022-08-23,644925.0,3231.0
366539,2022-08-24,644925.0,3231.0
366540,2022-08-25,644925.0,3231.0


In [332]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(cv,x='date', y='weekly_cases',labels={'ts': 'Время', 'prices': 'Цена'}, title='USA Weekly Cases',template = 'plotly_dark')
fig.update_traces(line=dict(color='cyan', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='black', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red',title = "Cases in week")  # Добавить сетку на оси y
fig.show()





In [333]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(cv,x='date', y='weekly_deaths',labels={'ts': 'Время', 'prices': 'Цена'}, title='USA Weekly Deaths',template = 'plotly_dark')
fig.update_traces(line=dict(color='cyan', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='black', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red',title = "Deaths in week")  # Добавить сетку на оси y
fig.show()





In [334]:
gt

,gtr
date,
2020-02-18,4.0
2020-02-19,4.0
2020-02-20,4.0
2020-02-21,4.0
2020-02-24,16.0
...,...
2022-08-22,0.0
2022-08-23,0.0
2022-08-24,0.0


In [337]:
# fig = px.line(sp, x='ts', y='prices', labels={'ts': 'Время', 'prices': 'Цена'}, title='График цены токена LIDO DAO')
fig = px.area(gt,labels={'ts': 'Время', 'prices': 'Цена'}, title='Coronavirus deaths google trend',template = 'plotly_dark')
fig.update_traces(line=dict(color='cyan', width=2))  # Изменить цвет и толщину линии
fig.update_layout(plot_bgcolor='black', title_font=dict(size=24))  # Изменить цвет фона и размер заголовка
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='red')  # Добавить сетку на оси x
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='red',title = "Value")  # Добавить сетку на оси y
fig.show()





In [348]:
price = np.array([
                  wmt_df["Close"].mean(),
                  wmt_df["Close"].std(),
                  wmt_df["Close"].min(),
                  wmt_df["Close"].quantile(0.25),
                  wmt_df["Close"].quantile(0.5),
                  wmt_df["Close"].quantile(0.75),
                  wmt_df["Close"].max()
 ])
volume = np.array([
                  wmt_df["Volume"].mean(),
                  wmt_df["Volume"].std(),
                  wmt_df["Volume"].min(),
                  wmt_df["Volume"].quantile(0.25),
                  wmt_df["Volume"].quantile(0.5),
                  wmt_df["Volume"].quantile(0.75),
                  wmt_df["Volume"].max()
 ])
Volt = np.array([
                  wmt_df["Volt"].mean(),
                  wmt_df["Volt"].std(),
                  wmt_df["Volt"].min(),
                  wmt_df["Volt"].quantile(0.25),
                  wmt_df["Volt"].quantile(0.5),
                  wmt_df["Volt"].quantile(0.75),
                  wmt_df["Volt"].max()
 ])
vol = np.array([conditional_volatility])
Gvolt= np.array([
                  vol.mean(),
                  vol.std(),
                  vol.min(),
                  np.quantile(vol,0.25),
                  np.quantile(vol,0.5),
                  np.quantile(vol,0.75),
                  vol.max()
 ])
illq= np.array([
                  wmt_df["illq"].mean(),
                  wmt_df["illq"].std(),
                  wmt_df["illq"].min(),
                  wmt_df["illq"].quantile(0.25),
                  wmt_df["illq"].quantile(0.5),
                  wmt_df["illq"].quantile(0.75),
                  wmt_df['illq'].max()
 ])

print(price,volume,Volt,Gvolt,illq,sep="\n")
########################################
price,volume,Volt,Gvolt,illq
sp500
cases,
deaths
string
gtrends

[45.82162269  3.48397746 34.68333435 43.79833412 46.63499832 48.22999954
 53.29000092]
[2.47135669e+07 1.33828449e+07 8.77740000e+06 1.71027750e+07
 2.07712500e+07 2.80941000e+07 1.32940200e+08]
[0.01835735 0.01151538 0.00520868 0.01161204 0.01556757 0.02157848
 0.1120802 ]
[0.05961238 0.02643957 0.04276573 0.04543277 0.05065339 0.06409335
 0.36891744]
[3.96367571e-03 3.16131376e-03 2.02101383e-05 1.63186059e-03
 3.34949698e-03 5.58522128e-03 3.68917443e-01]


array([  4.90076336,  14.65551245,   0.        ,   0.        ,
         0.        ,   2.        , 100.        ])

In [344]:
gtrends= np.array([
                  gtrend.mean(),
                  gtrend.std(),
                  gtrend.min(),
                  np.quantile(gtrend,0.25),
                  np.quantile(gtrend,0.5),
                  np.quantile(gtrend,0.75),
                  gtrend.max()
 ])

In [355]:
mobilities= np.array([
                  mobil['sum'].mean(),
                  mobil['sum'].std(),
                  mobil['sum'].min(),
                  mobil['sum'].quantile(0.25),
                  mobil['sum'].quantile(0.5),
                  mobil['sum'].quantile(0.75),
                  mobil['sum'].max()
 ])
mobilities


array([ -46.69749216,   42.80514457, -183.        ,  -73.        ,
        -39.        ,  -15.        ,   45.        ])

In [372]:
zoom= np.array([
                  zm['Zoom'].mean(),
                  zm['Zoom'].std(),
                  zm['Zoom'].min(),
                  zm['Zoom'].quantile(0.25),
                  zm['Zoom'].quantile(0.5),
                  zm['Zoom'].quantile(0.75),
                  zm['Zoom'].max()
 ])
names = np.array(['mean','std','min','25th percentile','mediana','75th percentile','max'])

In [386]:
data3 = pd.DataFrame({"Walmart":price,"Volume":volume,"Volt":Volt,"Gvolt":Gvolt,"Illiq":illq,
                    "S&P 500":sp500,"Cases":cases,"Deaths":deaths,"Stringency":string,"GTrends":gtrends,
                    "Mobilities":mobilities},index = names)
data3 = data3.round(2)
data3.T

,mean,std,min,25th percentile,mediana,75th percentile,max
Walmart,45.82,3.48,34.68,43.80,46.63,48.23,5.329000e+01
Volume,24713566.91,13382844.87,8777400.00,17102775.00,20771250.00,28094100.00,1.329402e+08
Volt,0.02,0.01,0.01,0.01,0.02,0.02,1.100000e-01
Gvolt,0.06,0.03,0.04,0.05,0.05,0.06,3.700000e-01
Illiq,0.00,0.00,0.00,0.00,0.00,0.01,3.700000e-01
S&P 500,0.00,0.02,-0.12,-0.01,0.00,0.01,9.000000e-02
Cases,597831.55,776321.67,0.00,222579.00,387634.00,709637.00,5.650933e+06
Deaths,6507.36,5505.61,0.00,2560.00,4493.00,9065.00,2.331200e+04
Stringency,38.96,19.70,0.00,22.84,31.69,55.09,9.352000e+01
GTrends,4.90,14.66,0.00,0.00,0.00,2.00,1.000000e+02
